In [1]:
#NOTE: use python env acmil in ACMIL folder
import sys
import os
import numpy as np
import openslide
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import warnings
import torch
import torch.nn as nn

from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import DataLoader, Subset, ConcatDataset
import torch.optim as optim
from pathlib import Path
import PIL
from skimage import filters
import random

    
sys.path.insert(0, '../Utils/')
from Utils import create_dir_if_not_exists
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords, get_map_startend
from Utils import get_downsample_factor
from Utils import minmax_normalize, set_seed
from Utils import log_message
from Eval import compute_performance, plot_LOSS, compute_performance_each_label, get_attention_and_tileinfo
from train_utils import pull_tiles
from train_utils import ModelReadyData_diffdim, convert_to_dict, prediction_sepatt, BCE_Weighted_Reg, BCE_Weighted_Reg_focal, compute_loss_for_all_labels_sepatt
from Model import Mutation_MIL_MT_sepAtt #, Mutation_MIL_MT
warnings.filterwarnings("ignore")
%matplotlib inline


#FOR ACMIL
current_dir = os.getcwd()
grandparent_subfolder = os.path.join(current_dir, '..', '..', 'other_model_code','ACMIL-main')
grandparent_subfolder = os.path.normpath(grandparent_subfolder)
sys.path.insert(0, grandparent_subfolder)
from architecture.transformer import ACMIL_GA
from utils.utils import save_model, Struct, set_seed
import yaml
import sys
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import yaml
from pprint import pprint

import argparse
import torch
from torch import nn
from torch.utils.data import DataLoader

from utils.utils import save_model, Struct, set_seed
from datasets.datasets import build_HDF5_feat_dataset
from architecture.transformer import ACMIL_GA #ACMIL_GA
from architecture.transformer import ACMIL_MHA
import torch.nn.functional as F

from utils.utils import MetricLogger, SmoothedValue, adjust_learning_rate
from timm.utils import accuracy
import torchmetrics
import wandb

In [2]:
####################################
######      USERINPUT       ########
####################################
SELECTED_LABEL = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
#SELECTED_LABEL = "MSI_POS"
TRAIN_SAMPLE_SIZE = "ALLTUMORTILES"
TRAIN_OVERLAP = 100
TEST_OVERLAP = 0
SELECTED_FOLD = 0
TUMOR_FRAC_THRES = 0.9
feature_extraction_method = 'retccl'
learning_method = "acmil"
INCLUDE_TF = False
INCLUDE_CLUSTER = False
N_CLUSTERS = 4
focal_gamma = 2


####
#model Para
LEARNING_RATE = 0.00001 
BATCH_SIZE  = 1
ACCUM_SIZE = 16  # Number of steps to accumulate gradients
EPOCHS = 100
DROPOUT = 0
DIM_OUT = 128

if INCLUDE_TF == False and INCLUDE_CLUSTER == False:
    N_FEATURE = 2048
elif INCLUDE_TF == True and INCLUDE_CLUSTER == False:
    N_FEATURE = 2049
elif INCLUDE_TF == False and INCLUDE_CLUSTER == True:
    N_FEATURE = 2049
elif INCLUDE_TF == True and INCLUDE_CLUSTER == True:
    N_FEATURE = 2050
            

LOSS_FUNC_NAME = "BCE_Weighted_Reg_focal" #"BCE_Weighted_Reg", "BCE_Weighted_Reg_focal"
REG_COEEF = 0.0000001
REG_TYPE = 'L1'
OPTMIZER = "ADAM"
ATT_REG_FLAG = False
SELECTED_MUTATION = "MT"

if SELECTED_MUTATION == "MT":
    N_LABELS = len(SELECTED_LABEL)
    LOSS_WEIGHTS_LIST = [[1, 50], [1, 100], [1, 50], [1, 100], [1, 100], [1, 10], [1, 30]]  #NEG, POS #WSI 50
else:
    N_LABELS = 1
    LOSS_WEIGHTS_LIST = [[1, 10], [1, 10], [1, 50], [1, 100], [1, 100], [1, 100], [1, 10]]  #NEG, POS

##################
###### DIR  ######
##################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
folder_name = feature_extraction_method + '/MAXSS'+ str(TRAIN_SAMPLE_SIZE)  + '_TrainOL' + str(TRAIN_OVERLAP) +  '_TestOL' + str(TEST_OVERLAP) + '_TFT' + str(TUMOR_FRAC_THRES) + "/split_fold" + str(SELECTED_FOLD) + "/" 
wsi_path = proj_dir + '/data/OPX/'
in_data_path = proj_dir + 'intermediate_data/model_ready_data/feature_' + folder_name + "model_input/"

if INCLUDE_TF == False and INCLUDE_CLUSTER == False:
    feature_type = "emb_only"
elif INCLUDE_TF == True and INCLUDE_CLUSTER == False:
    feature_type = "emb_and_tf"
elif INCLUDE_TF == False and INCLUDE_CLUSTER == True:
    feature_type = "emb_and_cluster" + str(N_CLUSTERS)
elif INCLUDE_TF == True and INCLUDE_CLUSTER == True:
    feature_type = "emb_and_tf_and_cluster" + str(N_CLUSTERS) 

model_data_path =  in_data_path + feature_type + "/"
    
################################################
#Create output-dir
################################################
outdir0 =  proj_dir + "intermediate_data/pred_out01212025" + folder_name + "/DL_" + feature_type + "/" + SELECTED_MUTATION + "/"
outdir1 =  outdir0  + "/saved_model/"
outdir2 =  outdir0  + "/model_para/"
outdir3 =  outdir0  + "/logs/"
outdir4 =  outdir0  + "/predictions/"
outdir5 =  outdir0  + "/perf/"
ckpt_dir = outdir1

create_dir_if_not_exists(outdir0)
create_dir_if_not_exists(outdir1)
create_dir_if_not_exists(outdir2)
create_dir_if_not_exists(outdir3)
create_dir_if_not_exists(outdir4)
create_dir_if_not_exists(outdir5)

##################
#Select GPU
##################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out01212025retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0//DL_emb_only/MT/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out01212025retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0//DL_emb_only/MT//saved_model/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out01212025retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0//DL_emb_only/MT//model_para/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out01212025retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0//DL_emb_only/MT//logs/' already exists.
Directory '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/intermediate_data/pred_out01212025retccl/MAXSSALLTUMORTILES_TrainOL100_TestOL0_TFT0.9/split_fold0//DL_emb_only/MT//predictions/' already exists.
Direct

In [3]:
################################################
#     Model ready data 
################################################
train_data_old = torch.load(model_data_path + 'train_data.pth')
test_data_old = torch.load(model_data_path + 'test_data.pth')
val_data = torch.load(model_data_path + 'val_data.pth')

train_ids_old = torch.load(model_data_path + 'train_ids.pth')
test_ids_old = torch.load(model_data_path + 'test_ids.pth')

train_info_old  = torch.load(model_data_path + 'train_info.pth')
test_info_old  = torch.load(model_data_path + 'test_info.pth')

new_data = torch.load(model_data_path + 'newMSI_test_data.pth')
new_ids = torch.load(model_data_path + 'newMSI_test_ids.pth')
new_info  = torch.load(model_data_path + 'newMSI_test_info.pth')

In [4]:
################################################
#Exclude OPX_085, Prostate cancer find in colorectal site, patterns are for CRC, not for prostate
################################################
exc_idx = test_ids_old.index('OPX_085')
inc_idx = [i for i in range(len(test_data_old)) if i not in [exc_idx]]

#Update old testset
test_data_old = Subset(test_data_old, inc_idx)
removed_id =   test_ids_old.pop(exc_idx)  
removed_info = test_info_old.pop(exc_idx)  

In [5]:
train_add_ids = ['OPX_207','OPX_209','OPX_213','OPX_214','OPX_215']
test_add_ids =  [x for x in new_ids if x not in train_add_ids]
print(train_add_ids)
print(test_add_ids)

['OPX_207', 'OPX_209', 'OPX_213', 'OPX_214', 'OPX_215']
['OPX_208', 'OPX_210', 'OPX_211', 'OPX_212', 'OPX_216']


In [6]:
################################################
#Add Ids in train 
################################################
inc_idx = [new_ids.index(x) for x in train_add_ids]
new_data_train = Subset(new_data, inc_idx)
new_id_train =  list(Subset(new_ids, inc_idx))
new_info_train = list(Subset(new_info, inc_idx))

#Combine old and new train data
train_data  = ConcatDataset([train_data_old, new_data_train])
train_ids = train_ids_old +  new_id_train
train_info = train_info_old +  new_info_train

In [7]:
################################################
#Add Ids in test 
################################################
inc_idx = [new_ids.index(x) for x in test_add_ids]
new_data_test = Subset(new_data, inc_idx)
new_id_test =  list(Subset(new_ids, inc_idx))
new_info_test = list(Subset(new_info, inc_idx))

#Combine old and new train data
test_data  = ConcatDataset([test_data_old, new_data_test])
test_ids = test_ids_old +  new_id_test
test_info = test_info_old +  new_info_test

In [8]:
#count labels in train
train_label_counts = [dt[1] for dt in train_data]
train_label_counts = torch.concat(train_label_counts)
count_ones = (train_label_counts == 1).sum(dim=0)
print(count_ones)
perc_ones = count_ones/train_label_counts.shape[0] * 100
formatted_numbers = [f"{x.item():.1f}" for x in perc_ones]
print(formatted_numbers)

tensor([15, 25, 31, 18, 58, 10,  9])
['9.9', '16.4', '20.4', '11.8', '38.2', '6.6', '5.9']


In [9]:
#count labels in test
test_label_counts = [dt[1] for dt in test_data]
test_label_counts = torch.concat(test_label_counts)
count_ones = (test_label_counts == 1).sum(dim=0)
print(count_ones)
perc_ones = count_ones/test_label_counts.shape[0] * 100
formatted_numbers = [f"{x.item():.1f}" for x in perc_ones]
print(formatted_numbers)

tensor([ 5,  6, 10,  4, 16,  7,  7])
['11.4', '13.6', '22.7', '9.1', '36.4', '15.9', '15.9']


In [10]:
####################################################
#            Train 
####################################################
set_seed(0)

#Dataloader for training
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)


#Construct model
# model = Mutation_MIL_MT_sepAtt(in_features = N_FEATURE, 
#                         act_func = 'tanh', 
#                         drop_out = DROPOUT,
#                         n_outcomes = N_LABELS,
#                         dim_out = DIM_OUT)

# model.to(device)

# #Optimizer
# if OPTMIZER == "ADAM":
#     optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# elif OPTMIZER == "SGD":
#     optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

# #Loss
# if LOSS_FUNC_NAME == "BCE_Weighted_Reg":
#     loss_func = BCE_Weighted_Reg(REG_COEEF, REG_TYPE, model, reduction = 'mean', att_reg_flag = ATT_REG_FLAG)
# elif LOSS_FUNC_NAME == "BCE_Weighted_Reg_focal":
#     loss_func = BCE_Weighted_Reg_focal(REG_COEEF, REG_TYPE, model, gamma = focal_gamma, reduction = 'mean', att_reg_flag = ATT_REG_FLAG)
# elif LOSS_FUNC_NAME == "BCELoss":
#     loss_func = torch.nn.BCELoss()
    

# #Model para
# total_params = sum(p.numel() for p in model.parameters())
# print(f"Number of parameters: {total_params}")
# #print(model)


#OUTPUT MODEL hyper-para
# hyper_df = pd.DataFrame({"Target_Mutation": SELECTED_MUTATION,
#                          "TRAIN_OVERLAP": TRAIN_OVERLAP,
#                          "TEST_OVERLAP": TEST_OVERLAP,
#                          "TRAIN_SAMPLE_SIZE": TRAIN_SAMPLE_SIZE,
#                          "TUMOR_FRAC_THRES": TUMOR_FRAC_THRES,
#                          "N_FEATURE": N_FEATURE,
#                          "N_LABELS": N_LABELS,
#                          "BATCH_SIZE": BATCH_SIZE,
#                          "ACCUM_SIZE": ACCUM_SIZE,
#                          "N_EPOCH": EPOCHS,
#                          "OPTMIZER": OPTMIZER,
#                          "LEARNING_RATE": LEARNING_RATE,
#                          "DROPOUT": DROPOUT,
#                          "DIM_OUT": DIM_OUT,
#                          "REG_TYPE": REG_TYPE,
#                          "REG_COEEF": REG_COEEF,
#                          "LOSS_FUNC_NAME": LOSS_FUNC_NAME,
#                          "LOSS_WEIGHTS_LIST": str(LOSS_WEIGHTS_LIST),
#                          "ATT_REG_FLAG": ATT_REG_FLAG,
#                          "NUM_MODEL_PARA": total_params}, index = [0])
# hyper_df.to_csv(outdir2 + "hyperpara_df.csv")

In [73]:
# get config
config_dir = "myconf.yml"
with open(config_dir, "r") as ymlfile:
    c = yaml.load(ymlfile, Loader=yaml.FullLoader)
    #c.update(vars(args))
    conf = Struct(**c)
    
conf.D_feat = N_FEATURE
conf.D_inner = DIM_OUT
conf.n_token = 5
conf.n_class = 2
conf.wandb_mode = 'disabled'


# Print all key-value pairs in the conf object
for key, value in conf.__dict__.items():
    print(f"{key}: {value}")

arch = 'ga'
# define network
if arch == 'ga':
    model = ACMIL_GA(conf, n_token=conf.n_token, n_masked_patch=0, mask_drop=0.6)
else:
    model = ACMIL_MHA(conf, n_token=conf.n_token, n_masked_patch=conf.n_masked_patch, mask_drop=conf.mask_drop)
model.to(device)


class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# Example usage:
criterion = FocalLoss(alpha=0.1, gamma=2, reduction='mean')
#criterion = nn.CrossEntropyLoss()

# define optimizer, lr not important at this point
optimizer0 = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=conf.wd)

train_epoch: 100
warmup_epoch: 0
wd: 1e-05
lr: 0.0001
min_lr: 0
dataset: bracs
B: 1
n_class: 2
data_dir: /mnt/Xsky/zyl/dataset/bracs/roi_feats_x100
n_worker: 8
pin_memory: False
n_shot: -1
backbone: ViT-S/16
pretrain: medical_ssl
D_feat: 2048
D_inner: 128
n_token: 5
wandb_mode: disabled


In [74]:
def train_one_epoch(model, criterion, data_loader, optimizer0, device, epoch, conf):
    """
    Trains the given network for one epoch according to given criterions (loss functions)
    """

    # Set the network to training mode
    model.train()

    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 100


    for data_it, data in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
        # for data_it, data in enumerate(data_loader, start=epoch * len(data_loader)):
        # Move input batch onto GPU if eager execution is enabled (default), else leave it on CPU
        # Data is a dict with keys `input` (patches) and `{task_name}` (labels for given task)
        image_patches = data[0].to(device, dtype=torch.float32)
        labels = data[1][0,:,6].to(device, dtype = torch.int64).to(device)

        # # Calculate and set new learning rate
        adjust_learning_rate(optimizer0, epoch + data_it/len(data_loader), conf)

        # Compute loss
        sub_preds, slide_preds, attn = model(image_patches)
        if conf.n_token > 1:
            loss0 = criterion(sub_preds, labels.repeat_interleave(conf.n_token))
        else:
            loss0 = torch.tensor(0.)
        loss1 = criterion(slide_preds, labels)


        diff_loss = torch.tensor(0).to(device, dtype=torch.float)
        attn = torch.softmax(attn, dim=-1)

        for i in range(conf.n_token):
            for j in range(i + 1, conf.n_token):
                diff_loss += torch.cosine_similarity(attn[:, i], attn[:, j], dim=-1).mean() / (
                            conf.n_token * (conf.n_token - 1) / 2)

        loss = diff_loss + loss0 + loss1

        optimizer0.zero_grad()
        # Backpropagate error and update parameters
        loss.backward()
        optimizer0.step()


        metric_logger.update(lr=optimizer0.param_groups[0]['lr'])
        metric_logger.update(sub_loss=loss0.item())
        metric_logger.update(diff_loss=diff_loss.item())
        metric_logger.update(slide_loss=loss1.item())

        if conf.wandb_mode != 'disabled':
            """ We use epoch_1000x as the x-axis in tensorboard.
            This calibrates different curves when batch size changes.
            """
            wandb.log({'sub_loss': loss0}, commit=False)
            wandb.log({'diff_loss': diff_loss}, commit=False)
            wandb.log({'slide_loss': loss1})

In [69]:
# Disable gradient calculation during evaluation
@torch.no_grad()
def evaluate(net, criterion, data_loader, device, conf, header):

    # Set the network to evaluation mode
    net.eval()

    y_pred = []
    y_true = []

    metric_logger = MetricLogger(delimiter="  ")

    for data in metric_logger.log_every(data_loader, 100, header):
        image_patches = data[0].to(device, dtype=torch.float32)
        labels = data[1][0,:,6].to(device, dtype = torch.int64).to(device)

        sub_preds, slide_preds, attn = net(image_patches)
        div_loss = torch.sum(F.softmax(attn, dim=-1) * F.log_softmax(attn, dim=-1)) / attn.shape[1]
        loss = criterion(slide_preds, labels)
        pred = torch.softmax(slide_preds, dim=-1)


        acc1 = accuracy(pred, labels, topk=(1,))[0]

        metric_logger.update(loss=loss.item())
        metric_logger.update(div_loss=div_loss.item())
        metric_logger.meters['acc1'].update(acc1.item(), n=labels.shape[0])

        y_pred.append(pred)
        y_true.append(labels)

    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)

    AUROC_metric = torchmetrics.AUROC(num_classes = conf.n_class, task='multiclass').to(device)
    AUROC_metric(y_pred, y_true)
    auroc = AUROC_metric.compute().item()
    F1_metric = torchmetrics.F1Score(num_classes = conf.n_class, task='multiclass').to(device)
    F1_metric(y_pred, y_true)
    f1_score = F1_metric.compute().item()

    print('* Acc@1 {top1.global_avg:.3f} loss {losses.global_avg:.3f} auroc {AUROC:.3f} f1_score {F1:.3f}'
          .format(top1=metric_logger.acc1, losses=metric_logger.loss, AUROC=auroc, F1=f1_score))

    return auroc, metric_logger.acc1.global_avg, f1_score, metric_logger.loss.global_avg

In [75]:
# define optimizer, lr not important at this point
optimizer0 = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=conf.wd)


best_state = {'epoch':-1, 'val_acc':0, 'val_auc':0, 'val_f1':0, 'test_acc':0, 'test_auc':0, 'test_f1':0}
train_epoch = conf.train_epoch
for epoch in range(train_epoch):
    train_one_epoch(model, criterion, train_loader, optimizer0, device, epoch, conf)


    val_auc, val_acc, val_f1, val_loss = evaluate(model, criterion, val_loader, device, conf, 'Val')
    test_auc, test_acc, test_f1, test_loss = evaluate(model, criterion, test_loader, device, conf, 'Test')

    if conf.wandb_mode != 'disabled':
        wandb.log({'perf/val_acc1': val_acc}, commit=False)
        wandb.log({'perf/val_auc': val_auc}, commit=False)
        wandb.log({'perf/val_f1': val_f1}, commit=False)
        wandb.log({'perf/val_loss': val_loss}, commit=False)
        wandb.log({'perf/test_acc1': test_acc}, commit=False)
        wandb.log({'perf/test_auc': test_auc}, commit=False)
        wandb.log({'perf/test_f1': test_f1}, commit=False)
        wandb.log({'perf/test_loss': test_loss}, commit=False)


    if val_f1 + val_auc > best_state['val_f1'] + best_state['val_auc']:
        best_state['epoch'] = epoch
        best_state['val_auc'] = val_auc
        best_state['val_acc'] = val_acc
        best_state['val_f1'] = val_f1
        best_state['test_auc'] = test_auc
        best_state['test_acc'] = test_acc
        best_state['test_f1'] = test_f1
        save_model(conf=conf, model=model, optimizer=optimizer0, epoch=epoch,
            save_path=os.path.join(ckpt_dir, 'checkpoint-best.pth'))
    print('\n')


save_model(conf=conf, model=model, optimizer=optimizer0, epoch=epoch,
    save_path=os.path.join(ckpt_dir, 'checkpoint-last.pth'))
print("Results on best epoch:")
print(best_state)
wandb.finish()

Epoch: [0]  [  0/152]  eta: 0:00:01  lr: 0.000100  sub_loss: 0.0197 (0.0197)  diff_loss: 1.0000 (1.0000)  slide_loss: 0.0163 (0.0163)  time: 0.0104  data: 0.0019  max mem: 3454
Epoch: [0]  [100/152]  eta: 0:00:00  lr: 0.000100  sub_loss: 0.0052 (0.0119)  diff_loss: 0.9981 (0.9995)  slide_loss: 0.0014 (0.0070)  time: 0.0178  data: 0.0035  max mem: 3454
Epoch: [0]  [151/152]  eta: 0:00:00  lr: 0.000100  sub_loss: 0.0031 (0.0117)  diff_loss: 0.9597 (0.9923)  slide_loss: 0.0011 (0.0093)  time: 0.0119  data: 0.0016  max mem: 3454
Epoch: [0] Total time: 0:00:02 (0.0141 s / it)
Val  [0/7]  eta: 0:00:00  loss: 0.0981 (0.0981)  div_loss: -6.4788 (-6.4788)  acc1: 0.0000 (0.0000)  time: 0.0014  data: 0.0003  max mem: 3454
Val  [6/7]  eta: 0:00:00  loss: 0.0010 (0.0149)  div_loss: -6.4788 (-6.9097)  acc1: 100.0000 (85.7143)  time: 0.0056  data: 0.0012  max mem: 3454
Val Total time: 0:00:00 (0.0057 s / it)
* Acc@1 85.714 loss 0.015 auroc 0.667 f1_score 0.857
Test  [ 0/44]  eta: 0:00:00  loss: 0.001

In [20]:
first_batch = next(iter(test_loader))
feat = first_batch[0].to(device)
sub_preds, slide_preds, attn = model(feat)
slide_preds

tensor([[ 1.4293, -1.7806]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [77]:
# Set the network to evaluation mode
model.eval()

y_predprob = []
y_pred = []
y_true = []

metric_logger = MetricLogger(delimiter="  ")

for data in metric_logger.log_every(test_loader, 100, None):
    image_patches = data[0].to(device, dtype=torch.float32)
    labels = data[1][0,:,6].to(device, dtype = torch.int64).to(device)

    sub_preds, slide_preds, attn = model(image_patches)
    pred = torch.softmax(slide_preds, dim=-1)
    pred_prob = torch.softmax(slide_preds, dim=-1)[:,1]

    y_predprob.append(pred_prob)
    y_pred.append(pred)
    y_true.append(labels)
    
y_predprob = torch.cat(y_predprob, dim=0)
y_pred = torch.cat(y_pred, dim=0)
y_true = torch.cat(y_true, dim=0)

AUROC_metric = torchmetrics.AUROC(num_classes = conf.n_class, task='multiclass').to(device)
AUROC_metric(y_pred, y_true)
auroc = AUROC_metric.compute().item()
F1_metric = torchmetrics.F1Score(num_classes = conf.n_class, task='multiclass').to(device)
F1_metric(y_pred, y_true)
f1_score = F1_metric.compute().item()
print(auroc)
print(f1_score)

  [ 0/44]  eta: 0:00:00    time: 0.0099  data: 0.0051  max mem: 3454
  [43/44]  eta: 0:00:00    time: 0.0034  data: 0.0008  max mem: 3454
 Total time: 0:00:00 (0.0029 s / it)
0.7799227833747864
0.8409090638160706


In [78]:
####################################################################################
#Predict
####################################################################################

#predicts
test_pred_prob  = y_predprob
test_true_label = y_true

#Prediction df
pred_df = pd.DataFrame({"SAMPLE_IDs":  test_ids, 
                            "Y_True": y_true.cpu().detach().numpy(), 
                            "Pred_Prob" :  test_pred_prob.cpu().detach().numpy(),
                            "OUTCOME": 'MSI_POS'})

#Add Predict class
THRES = 0.2
pred_df['Pred_Class'] = 0
pred_df.loc[pred_df['Pred_Prob'] > THRES,'Pred_Class'] = 1
pred_df.to_csv(outdir4 + "/pred_df.csv",index = False)


# #Compute performance
perf_df = compute_performance_each_label(['MSI_POS'], pred_df, "SAMPLE_LEVEL")
print(perf_df)
# perf_df.to_csv(outdir5 + "/perf.csv",index = True)

# print(perf_df.iloc[:,[0,5,6,7,8,9]])
# print("AVG AUC:", round(perf_df['AUC'].mean(),2))
# print("AVG PRAUC:", round(perf_df['PR_AUC'].mean(),2))
# #Use regularization no dropout now has the best performance at avg AUC = 0.61

               AUC   ACC    F1    F2    F3  Recall  Precision  Specificity  \
SAMPLE_LEVEL  0.78  0.55  0.38  0.57  0.68    0.86       0.24         0.49   

                PR_AUC  OUTCOME  
SAMPLE_LEVEL  0.501127  MSI_POS  


In [80]:
test_pred_prob

tensor([0.1333, 0.2903, 0.2327, 0.2121, 0.2469, 0.3024, 0.2164, 0.1665, 0.3794,
        0.2292, 0.1358, 0.3845, 0.1216, 0.2009, 0.3400, 0.1800, 0.2402, 0.2324,
        0.2339, 0.2299, 0.1385, 0.1651, 0.1929, 0.3441, 0.1444, 0.2765, 0.2272,
        0.1677, 0.2151, 0.1068, 0.1047, 0.1515, 0.1805, 0.1039, 0.1333, 0.1762,
        0.1840, 0.1680, 0.2336, 0.2122, 0.2479, 0.3233, 0.3481, 0.2679],
       device='cuda:0', grad_fn=<CatBackward0>)

In [ ]:
# ####################################################################################
# #Atention scores
# ####################################################################################
# save_image_size = 250
# pixel_overlap = 0
# mag_extract = 20
# limit_bounds = True
# TOP_K = 5
# pretrain_model_name = "retccl"
# mag_target_prob = 2.5
# smooth = False
# mag_target_tiss = 1.25
# #SELECTED_LABEL = ["AR","MMR (MSH2, MSH6, PMS2, MLH1, MSH3, MLH3, EPCAM)2","PTEN","RB1","TP53","TMB_HIGHorINTERMEDITATE","MSI_POS"]
# #selected_outcome = 'RB1'
# #selected_att_index = SELECTED_LABEL.index(selected_outcome)
# selected_type = "true_pos"

# # for selected_outcome in SELECTED_LABEL:
# for selected_outcome in SELECTED_LABEL:
#     selected_att_index = SELECTED_LABEL.index(selected_outcome)

#     selected_ids = true_postive_ids['MSI_POS']
#     for tp_pt in selected_ids:
#         i = test_ids.index(tp_pt)
#         pt = test_ids[i]
#         print(pt)
    
#         save_location = outdir4 + selected_outcome + "/" + selected_type + "/"
#         save_location =  save_location  + pt + "/"
#         create_dir_if_not_exists(save_location)
        
#         _file = wsi_path + pt + ".tif"
#         oslide = openslide.OpenSlide(_file)
#         save_name = str(Path(os.path.basename(_file)).with_suffix(''))
        
        
#         #Get a Attention, and corresponding tiles
#         cur_pt_att = test_att[i][selected_att_index]
#         cur_pt_info = test_info[i]
#         cur_att_df = get_attention_and_tileinfo(cur_pt_info, cur_pt_att)
        
        
#         #Generate tiles
#         tiles, tile_lvls, physSize, base_mag = generate_deepzoom_tiles(oslide,save_image_size, pixel_overlap, limit_bounds)
        
#         #get level 0 size in px
#         l0_w = oslide.level_dimensions[0][0]
#         l0_h = oslide.level_dimensions[0][1]
        
#         #1.25x tissue detection for mask
#         from Utils import get_downsample_factor, get_image_at_target_mag
#         from Utils import do_mask_original,check_tissue,whitespace_check
#         import cv2
#         if 'OPX' in pt:
#             rad_tissue = 5
#         elif '(2017-0133)' in pt:
#             rad_tissue = 2
#         lvl_resize_tissue = get_downsample_factor(base_mag,target_magnification = mag_target_tiss) #downsample factor
#         lvl_img = get_image_at_target_mag(oslide,l0_w, l0_h,lvl_resize_tissue)
#         tissue, he_mask = do_mask_original(lvl_img, lvl_resize_tissue, rad = rad_tissue)
        
#         #2.5x for probability maps
#         lvl_resize = get_downsample_factor(base_mag,target_magnification = mag_target_prob) #downsample factor
#         x_map = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
#         x_count = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
        
#         cur_att_df['pred_map_location'] = pd.NA
#         for index, row in cur_att_df.iterrows():
#             cur_xy = row['TILE_XY_INDEXES'].strip("()").split(", ")
#             x ,y = int(cur_xy[0]) , int(cur_xy[1])
            
#             #Extract tile for prediction
#             lvl_in_deepzoom = tile_lvls.index(mag_extract)
#             tile_starts, tile_ends, save_coords, tile_coords = extract_tile_start_end_coords(tiles, lvl_in_deepzoom, x, y) #get tile coords
#             map_xstart, map_xend, map_ystart, map_yend = get_map_startend(tile_starts,tile_ends,lvl_resize) #Get current tile position in map
#             cur_att_df.loc[index,'pred_map_location'] = str(tuple([map_xstart, map_xend, map_ystart, map_yend]))
        
#             #Store predicted probabily in map and count
#             try: 
#                 x_count[map_xstart:map_xend,map_ystart:map_yend] += 1
#                 x_map[map_xstart:map_xend,map_ystart:map_yend] += row['ATT']
#             except:
#                 pass
        
#         print('post-processing')
#         x_count = np.where(x_count < 1, 1, x_count)
#         x_map = x_map / x_count
#         x_map[x_map>1]=1
        
#         if smooth == True:
#             x_sm = filters.gaussian(x_map, sigma=2)
#         if smooth == False:
#             x_sm = x_map
        
#         he_mask = cv2.resize(np.uint8(he_mask),(x_sm.shape[1],x_sm.shape[0])) #resize to output image size
#         #TODO:
#         #get cancer_mask:
#         # cancer_mask == 
#         # x_sm[(he_mask == 1) & (x_sm == 0)] = 0.1 #If tissue map value > 1, then x_sm = 1
#         x_sm[he_mask < 1] = 0.001 
        
#         plt.imshow(x_sm, cmap='Spectral_r')
#         plt.colorbar()
#         plt.savefig(os.path.join(save_location, save_name + '_attention.png'), dpi=500,bbox_inches='tight')
#         plt.show()
#         plt.close()
        
        
#         #Top attented tiles
#         save_location2 = save_location + "top_tiles/"
#         create_dir_if_not_exists(save_location2)
        
#         #Get a Attention, and corresponding tiles
#         cur_att_df= cur_att_df.sort_values(by = ['ATT'], ascending = False) 
#         cur_pulled_img_obj = pull_tiles(cur_att_df.iloc[0:TOP_K], tiles, tile_lvls)
        
#         for i in range(TOP_K):
#             cur_pulled_img = cur_pulled_img_obj[i][0] #image
#             cur_pulled_att = cur_pulled_img_obj[i][1] #attentiom
#             cur_pulled_coord = cur_pulled_img_obj[i][2].strip("()").split(", ")  #att tile map coordiates
#             coord_save_name = '[xs' + cur_pulled_coord[0] + '_xe' + cur_pulled_coord[1] + '_ys' + cur_pulled_coord[2] + '_ye' + cur_pulled_coord[3] + "]"
#             tile_save_name = "ATT" + str(round(cur_pulled_att,2)) + "_MAPCOORD" +  coord_save_name +  ".png"
#             cur_pulled_img.save(os.path.join(save_location2, tile_save_name))
        
#         #Bot attented tiles
#         save_location2 = save_location + "bot_tiles/"
#         create_dir_if_not_exists(save_location2)
        
#         #Get a Attention, and corresponding tiles
#         cur_att_df= cur_att_df.sort_values(by = ['ATT'], ascending = True) 
#         cur_pulled_img_obj = pull_tiles(cur_att_df.iloc[0:TOP_K], tiles, tile_lvls)
        
#         for i in range(TOP_K):
#             cur_pulled_img = cur_pulled_img_obj[i][0] #image
#             cur_pulled_att = cur_pulled_img_obj[i][1] #attentiom
#             cur_pulled_coord = cur_pulled_img_obj[i][2].strip("()").split(", ")  #att tile map coordiates
#             coord_save_name = '[xs' + cur_pulled_coord[0] + '_xe' + cur_pulled_coord[1] + '_ys' + cur_pulled_coord[2] + '_ye' + cur_pulled_coord[3] + "]"
#             tile_save_name = "ATT" + str(round(cur_pulled_att,2)) + "_MAPCOORD" +  coord_save_name +  ".png"
#             cur_pulled_img.save(os.path.join(save_location2, tile_save_name))